# Results

In [32]:
##### PARAMS:
RESULTS_5M_FILENAME = './results/btc5min_c.csv'
RESULTS_5M_OHLC_FILENAME = './results/btc5min_ohlc.csv'
RESULTS_5M_OHLC_DOW_FILENAME = './results/btc5min_ohlc_dow.csv'

In [3]:
# autoreload
%load_ext autoreload
%autoreload 1
# write code that works in both Python 2 (2.7+) and 3 (3.5+)
from __future__ import absolute_import, division, print_function, unicode_literals
try:
    input = raw_input
    range = xrange
except NameError:
    pass

In [4]:
import csv
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import keras
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

import pprint
pp = pprint.PrettyPrinter(indent=2).pprint
from IPython.display import display as dd
from IPython.display import Image

%matplotlib inline

Using TensorFlow backend.


In [5]:
%aimport lib.helpers
%aimport lib.etl
%aimport lib.visualization
%aimport lib.walk_forward_predict
%aimport models.rnn
%aimport models.lr
from lib.helpers import (
    timing, print_shapes, fix_all_rngs,
    date_str_to_ts
)
from lib.visualization import simple_plot
from lib.walk_forward_predict import (
    run_walk_forward_validation_rnn,
    run_walk_forward_validation_arima
)
import lib.etl as etl
from models.rnn import make_rnn_model_constructor
from models.lr import LRModel

In [26]:
def summarize_results(filename):
    results = pd.read_csv(filename)
    print("> # rows in file %s: %d" % (
        filename, len(results)))
#     dd(results.dtypes)
#     dd(results.head())
#     dd(results.tail())
    results['dir_acc'] = results.dir_acc.str.replace('%', '').astype(float)
    print("> whole data dir_acc mean: %.2f%%" % results.dir_acc.mean())
    print("> last 50%% dir_acc mean: %.2f%%" % results[int(len(results) / 2):].dir_acc.mean())
    print("> last 25%% dir_acc mean: %.2f%%" % results[int(len(results) * 3 / 4):].dir_acc.mean())
    print("> last 10%% dir_acc mean: %.2f%%" % results[int(len(results) * 0.9):].dir_acc.mean())

## Goals recap

### 24 H

Data at less than 2 point prediction into the future is too noisy.

```
Min acc at 7: 0.5167313590715515
Min acc at 5: 0.5223033374369233
Min acc at 4: 0.5277445477717878
Min acc at 3: 0.5410135624970198
```

We might assume that a >60% at 1 day ahead accurate model has chance of driving a profitable strategy. Using 5min resolution data (see below) to calculate required accuracy for a 1 day ahead prediction could day that even >55% can be enough.

Taking an average, >57.5% at one day ahead could be taken as the minimal accuracy that has a change of driving a profitable trading strategy.

At 7 days ahead it might be enough to go >52%.

### 5 Min

* at 50 points (4.17h) ahead: >64%
* at 75 points (6.25h) ahead: >61%
* at 100 points (8.33h) ahead: >59%
* at 288 points (1 day) ahead: >55%

## 5 Min

In [27]:
summarize_results(RESULTS_5M_FILENAME)

> # rows in file ./results/btc5min_c.csv: 121
> whole data dir_acc mean: 56.37%
> last 50% dir_acc mean: 55.68%
> last 25% dir_acc mean: 55.80%
> last 10% dir_acc mean: 57.91%


In [30]:
summarize_results(RESULTS_5M_OHLC_FILENAME)

> # rows in file ./results/btc5min_ohlc.csv: 120
> whole data dir_acc mean: 46.52%
> last 50% dir_acc mean: 43.36%
> last 25% dir_acc mean: 42.65%
> last 10% dir_acc mean: 45.57%


In [34]:
summarize_results(RESULTS_5M_OHLC_DOW_FILENAME)

> # rows in file ./results/btc5min_ohlc_dow.csv: 120
> whole data dir_acc mean: 48.40%
> last 50% dir_acc mean: 45.27%
> last 25% dir_acc mean: 45.92%
> last 10% dir_acc mean: 51.92%


## 24 H